# Regression and Data Analysis in Python
This tutorial focuses on some basic tools for regression analysis and data visualization available in Python. The main tools are Pandas and StatsModels. Pandas is a powerful data storage engine built in a combination of C++ (for speed) and Python (for accessiblity). StatsModels is a Python package developed by a group of econometricians that implements a variety of regression and discrete choice models. We will also show some basic plotting functionalities using matplotlib.

The dataset used in this tutorial is Q4 bikeshare data for the City of Toronto. We link this data with weather data to provide additional explanatory variables. The data is essentially imported to Python with no pre-processing to show the ability of Python to handle these tasks. We must first import the necessary Python packages.

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
import seaborn as sns
sns.set()  # use Seaborn styles. 

We can now import the two datasets from csv files using Pandas. This package can also import STATA, shapefile, and a wide variety of other formats with minimal effort.

In [ ]:
dfB = pd.read_csv('./data/2016_Bike_Share_Toronto_Ridership_Q4.csv',sep=',')
dfW = pd.read_csv('./data/weather_2016_Q4.csv',sep=',')
dfB.head()

In [ ]:
dfB.tail()

It is clear that the 'trip_start_time' column in the bikeshare data does not have a consistent data-time format. We also want to breakup the date and time into components to facilitate easy of data analysis. This could be done in a spreadsheet, but Pandas provides a way to convert the data into a consistent datetime format, even with multiple input datetime formats (as in our case). The same process can be used for the weather data.

In [33]:
dfB.loc[:, 'trip_start_time_dt'] = pd.to_datetime(dfB['trip_start_time'], format='mixed')
dfB.loc[:, 'trip_stop_time_dt'] = pd.to_datetime(dfB['trip_stop_time'], format='mixed')
dfW.loc[:, 'date_time_dt'] = pd.to_datetime(dfW['date_time'])

Once thde data are converted to a consistent datetime format, the indivual components can be put in their own columns within the Pandas dataframe.

In [34]:
dfB.loc[:, 'trip_month'] = dfB['trip_start_time_dt'].dt.month
dfB.loc[:, 'trip_start_hour'] = dfB['trip_start_time_dt'].dt.hour
dfB.loc[:, 'day_of_week'] = dfB['trip_start_time_dt'].dt.dayofweek
dfB.loc[:, 'trip_date'] = dfB['trip_start_time_dt'].dt.date
dfW.loc[:, 'weather_date'] = dfW['date_time_dt'].dt.date

Pandas includes a variety of dataframe joining methods. The .merge() function functions similar to a database operation, allowing for left, right, inner, and outer joins. In the next line of code, we join the bikeshare and weather data on the date.

In [ ]:
df = pd.merge(dfB, dfW, left_on='trip_date', right_on='weather_date')
df = df.dropna(subset=['trip_start_hour','temp','wind_spd'])
df.head()

Our new dataframe has the following columns.

In [ ]:
df.columns

## Basic OLS model

Writing models using StatsModels is quite straightforward. It is made easier using the patsy function syntax used below. This includes the ability to write regression models in standard text using the column names from the dataframe.

StatsModels provides an extensive output, including the R-squared, AIC, BIC, and log likelihood. The R-squared in this first model is very low. We can probably do better.

In [ ]:
mod = smf.ols(formula='trip_duration_seconds ~ trip_start_hour + temp + wind_spd', data=df)
res = mod.fit()
print(res.summary())

The patsy syntax allows us to define categorical variables by simply enclosing a dataframe column name in C(). We include additional user, temporal, and weather data in this model. It is likely that trip length will vary by the weather and user type.

In [ ]:
mod = smf.ols(formula='trip_duration_seconds ~ C(user_type) + C(day_of_week) + trip_start_hour + temp + wind_spd + rel_hum', data=df)
res = mod.fit()
print(res.summary())

This is a better fit, but still not fantastic. We can print statistics on the dataframe using the .describe() function.

In [ ]:
df.describe()

In [ ]:
plt.hist(df['trip_start_hour'], len(df['trip_start_hour'].unique()), density=True, facecolor='g', alpha=0.75)
plt.xlabel('Start Hour');
plt.ylabel('Frequency');
plt.title('Histogram of Trip Start Hour');

In [ ]:
plt.hist(df['day_of_week'], len(df['day_of_week'].unique()), density=True, facecolor='b', alpha=0.75)
plt.xlabel('Day of Week');
plt.ylabel('Frequency');
plt.title('Histogram of Trip Day');

## Multicollinearity - Condition number
One way to assess multicollinearity is to compute the condition number. Values over 20 are worrisome (see Greene 4.9). The first step is to normalize the independent variables to have unit length.

In [42]:
names = ['trip_start_hour', 'temp', 'wind_spd', 'rel_hum']
X = df.loc[:, names]
norm_x = X.values
for i, name in enumerate(X):
    if name == "const":
        continue
    norm_x[:, i] = X[name] / np.linalg.norm(X[name])
norm_xtx = np.dot(norm_x.T, norm_x)

Then, we take the square root of the ratio of the biggest to the smallest eigen values.

In [ ]:
eigs = np.linalg.eigvals(norm_xtx)
condition_number = np.sqrt(eigs.max() / eigs.min())
print(condition_number)

It seems multicollinearity is not an issue here.

## Normality of residuals
### Jarque-Bera test

In [ ]:
name = ["Jarque-Bera", "Chi^2 two-tail prob.", "Skew", "Kurtosis"]
test = sms.jarque_bera(res.resid)
lzip(name, test)

### Omni test

In [ ]:
name = ["Chi^2", "Two-tail probability"]
test = sms.omni_normtest(res.resid)
lzip(name, test)

Residuals are not normally distributed for this model.

## Heteroskedasticity tests
### Breush-Pagan test

In [ ]:
name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
test = sms.het_breuschpagan(res.resid, res.model.exog)
lzip(name, test)

### Goldfeld-Quandt test

In [ ]:
name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(res.resid, res.model.exog)
lzip(name, test)

The Breush-Pagan test suggests the residuals are heteroskedastic but the Goldfeld-Quandt suggests they are not. It is not uncommon to get conflicting results from tests.